This is a tutorial to use the flip package: https://github.com/corentinravoux/flip \
It is self-contained and can be used in google collab or on your environement \
All the data used are subsampled version of a simulation. \
The data size is small for the tutorial, do not use it for science case. \

In [ ]:
%%capture
!pip install git+https://github.com/corentinravoux/flip

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from flip import utils, fitter, plot_utils, data_vector, __flip_dir_path__
from flip.covariance import covariance, contraction
import flip
flip_base = Path(__flip_dir_path__)
data_path = flip_base / "data"
plt.style.use(data_path / "style.mplstyle")

[ 000000.00 ]: 11-24 14:45  root            WARNING  Install pytorch to use the nnmatrix emulator
[ 000000.00 ]: 11-24 14:45  root            INFO     No pypower module detected, gridding with this method is unavailable
[ 000000.00 ]: 11-24 14:45  root            WARNING  Install CLASS https://github.com/lesgourg/class_public to use class_engine.py module
[ 000000.00 ]: 11-24 14:45  root            WARNING  Install CCL https://github.com/LSSTDESC/CCL to use pyccl_engine.py module
[ 000000.00 ]: 11-24 14:45  root            INFO     Jax is not available, loading numpy and scipy instead


In [2]:
flip.covariance.emulators.gpmatrix.gpy_installed

True

Loading the data, located in the package itself



In [3]:
data_velocity = pd.read_parquet(data_path / "velocity_data.parquet")


data_velocity_true = {**data_velocity.rename(columns={'vpec': 'velocity'}).to_dict(orient='list'), 
                      "velocity_error": np.zeros(len(data_velocity["vpec"].values))}


# Create the data Vector
DataTrueVel = data_vector.DirectVel(data_velocity_true)

ktt, ptt = np.loadtxt(data_path / "power_spectrum_tt.txt")
kmt, pmt = np.loadtxt(data_path / "power_spectrum_mt.txt")
kmm, pmm = np.loadtxt(data_path / "power_spectrum_mm.txt")

sigmau_fiducial = 15

power_spectrum_dict = {"vv": [[ktt, ptt * utils.Du(ktt, sigmau_fiducial) ** 2]]}


# Fit true velocity interpolation sigma_u

In the model, there is a sigma_u parameter which can be interpreted to the smoothing of SNIa position due to peculiar velocities. \
To fit simultaneously this parameter, we interpolate the covariance matrix. 

In [4]:
sigmau_list = np.linspace(10.0, 20.0,20)[:, np.newaxis]
covariance_list = []
size_batch=10_000
number_worker=16


for sigu in sigmau_list:
    
    power_spectrum_dict = {"vv": [[ktt, ptt* utils.Du(ktt, sigu)**2]]}

    covariance_list.append(
        DataTrueVel.compute_covariance(
            "carreres23",
            power_spectrum_dict,
            size_batch=size_batch,
            number_worker=number_worker
        )
    )


[ 000012.08 ]: 11-24 14:45  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.77e+00 seconds
[ 000013.97 ]: 11-24 14:45  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.72e+00 seconds
[ 000015.85 ]: 11-24 14:45  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.70e+00 seconds
[ 000017.78 ]: 11-24 14:45  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.76e+00 seconds
[ 000019.91 ]: 11-24 14:45  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.90e+00 seconds
[ 000022.02 ]: 11-24 14:45  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.90e+00 seconds
[ 000024.13 ]: 11-24 14:45  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.89e+00 seconds
[ 000026.26 ]: 11-24 14:45  root            INFO     Covariance matri

In [7]:
covariance_list

In [5]:
emulator_model = "gpmatrix"
emulator_parameter_names = ["sigu"]

covariance_gpmatrix = covariance.CovMatrix.init_from_emulator(
            emulator_model,
            "velocity",
            covariance_list,
            sigmau_list,
            emulator_parameter_names,
            verbose=True,
            test_emulator=False,
        )



[ 000067.35 ]: 11-24 14:46  GP              INFO     initializing Y
[ 000067.35 ]: 11-24 14:46  GP              INFO     initializing inference method
[ 000067.35 ]: 11-24 14:46  GP              INFO     adding kernel and likelihood as parameters


Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s01  0003   1.986109e+08   3.659187e+16 
    00s05  0010   1.105949e+07   4.594827e+12 
    00s10  0015   8.905160e+05   1.419973e+10 
    00s14  0021   2.562922e+04   2.871274e+06 
    00s17  0025   3.415685e+03   1.791383e+04 
    00s19  0029   5.254521e+02   1.137233e+02 
    00s43  0097   8.086316e+01   1.826155e-01 

[ 000067.84 ]: 11-24 14:46  GP              INFO     initializing Y
[ 000067.84 ]: 11-24 14:46  GP              INFO     initializing inference method
[ 000067.84 ]: 11-24 14:46  GP              INFO     adding kernel and likelihood as parameters



Runtime:     00s43
Optimization status: ErrorABNORMAL_TERMINATION_IN_LNSRCH

Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    05s07  0005   1.546440e+12   1.111035e+24 
    06s09  0006   1.031209e+12   3.078265e+23 
    15s03  0015   2.142431e+10   9.444231e+18 
    21s98  0022   4.371481e+08   8.274744e+14 
    34s07  0034   9.906507e+07   6.074124e+10 
    48s12  0048   9.774603e+07   5.984039e+05 
 01m20s75  0081   9.774600e+07   5.879689e+05 
Runtime:  01m20s75
Optimization status: ErrorABNORMAL_TERMINATION_IN_LNSRCH



[ 000150.75 ]: 11-24 14:47  root            INFO     Covariance matrix generated from emulator with gpmatrix model in 8.51e+01 seconds


In [6]:
parameter_dict = {"fs8": {"value": 0.4,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "sigv": {"value": 200,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "sigu": {"value": 15.5,
                          "limit_low" : 10.0,
                          "limit_up" : 20.0,
                          "fixed" : False,
                         },
                 }


prior_dict = {"sigu": {"type": "gaussian",
                       "mean" : 15.0,
                       "standard_deviation" : 5.0,},
             }

likelihood_type="multivariate_gaussian"
likelihood_properties = {"inversion_method": "cholesky",
                         "prior": prior_dict}



minuit_fitter = fitter.FitMinuit.init_from_covariance(
        covariance_gpmatrix,
        DataTrueVel,
        parameter_dict,
        likelihood_type=likelihood_type,
        likelihood_properties = likelihood_properties,
)

print('Free parameters:', minuit_fitter.likelihood.free_par)

minuit_fitter.run()


Free parameters: ['fs8', 'sigv']


AttributeError: 'list' object has no attribute 'shape'